In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data=pd.read_csv('/kaggle/input/ethereum-data/ETH-USD.csv')

In [ ]:
data.describe()

In [ ]:
data.dtypes

In [ ]:
data['Date'] = pd.to_datetime(data['Date'],format ='%Y-%m-%d' )

In [ ]:
data.tail()

In [ ]:
data['mm5d'] = data['Close'].rolling(5).mean()
data['mm21d'] = data['Close'].rolling(21).mean()

In [ ]:
data['Close'] = data['Close'].shift(-1)
data.head()

In [ ]:
data.dropna(inplace=True)
data = data.reset_index(drop=True)
data.head()

In [ ]:
# train vs test and validation data
nlines = len(data)

qtd_linhas_treino= round(.70 * nlines)
qtd_linhas_teste= nlines - qtd_linhas_treino  
qtd_linhas_validacao = qtd_linhas_treino - qtd_linhas_teste

info = (
    f"train rows = 0:{qtd_linhas_treino}"
    f" test rows = {qtd_linhas_treino}:{qtd_linhas_treino + qtd_linhas_teste -1}"
    f" validation rows = {qtd_linhas_teste}:{nlines-1}"
)

info

In [ ]:
features = data.drop(['Date','Close'],1)
labels = data['Close']

In [ ]:
features_list = ('Open','High','Low','Close','Adj Close','Volume','mm5d','mm21d')

k_best_features = SelectKBest(k='all')
k_best_features.fit_transform(features, labels)
k_best_features_scores = k_best_features.scores_
raw_pairs = zip(features_list[1:], k_best_features_scores)
ordered_pairs = list(reversed(sorted(raw_pairs, key=lambda x: x[1])))

k_best_features_final = dict(ordered_pairs[:15])
best_features = k_best_features_final.keys()
print ('')
print ("Best features:")
print (k_best_features_final)

In [ ]:
features = data.loc[:,['Low','Adj Close','mm21d','Close','mm21d','High']]

In [ ]:
X_train = features[:qtd_linhas_treino]
X_test = features[qtd_linhas_treino:qtd_linhas_treino + qtd_linhas_teste -1]

y_train = labels[:qtd_linhas_treino]
y_test = labels[qtd_linhas_treino:qtd_linhas_treino + qtd_linhas_teste -1]

print( len(X_train), len(y_train))

print( len(X_test), len(y_test))

In [ ]:
scaler = MinMaxScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.fit_transform(X_test)

In [ ]:
lr = linear_model.LinearRegression()
lr.fit(X_train_scale, y_train)
pred= lr.predict(X_test_scale)
cd =r2_score(y_test, pred)

f'Coeficiente de determinação:{cd * 100:.2f}'

In [ ]:
#rede neural
rn = MLPRegressor(max_iter=2000)

rn.fit(X_train_scale, y_train)
pred= rn.predict(X_test_scale)

cd = rn.score(X_test_scale, y_test)


f'Coeficiente de determinação:{cd * 100:.2f}'